In [53]:
from bayes_opt import BayesianOptimization
import pandas as pd
import random, os

In [54]:
random.seed(5835)
import RF_abstraction

In [55]:
PATH = os.getcwd()
PATH = PATH.split(os.path.sep)
if "src" in PATH:
    PATH.remove("src")
PATH = os.path.sep.join(PATH)

def df_preprocessing(data_name: str) -> pd.DataFrame:
    df = pd.read_csv(os.path.join(PATH, "datasets", data_name + "_dataset.csv"))
    # (A) There are multiple of the same experiments, so average them!
    features = list(df.columns)[:-1]
    obj_metric = list(df.columns)[-1]
    df = df.groupby(features)[obj_metric].agg(lambda x: x.unique().mean())
    df = (df.to_frame()).reset_index()
    # (B) Change all NECESSARY df to a maximization problem
    if data_name not in ['P3HT', 'Crossed barrel', 'AutoAM']:
        # aka AgNP & Perovskite
        df[obj_metric] = -df[obj_metric].values

In [56]:
n_models = 50
n_models = [random.randint(0, 9999) for _ in range(n_models)]
# include 2 initial points somehow

What I notice is that in the research paper's code, they go through all possible candidates and run the the acquisition functions on each to pick the point with the largest acquisition value.
Its hard to do that in this library, instead...

In [59]:
dataset_df = df_preprocessing('Crossed barrel')


template_model = RF_abstraction.RF(
    acquisition_type= "EI",
    n_est = 100,
    ucb_ratio = 10,
    df = dataset_df,
    n_top_candidates = 0.05,
    seed = 5835)


for seed in n_models:
    Model = template_model.copy()
    Model.seed = seed
    
    optimizer = BayesianOptimization(
        f = Model.f_surrogate_model,
        pbounds = Model.pbounds,
        random_state= Model.seed
    )
    
    while Model.calc_metric != 1.0:
        next_point = optimizer.suggest(utility_function=Model.utility)
        Model.add_candidate(**next_point)
        value = Model.f_surrogate_model(**next_point)
        optimizer.register(params=next_point, target=value)

TypeError: RF.__init__() got an unexpected keyword argument 'n_est'